In [1]:
cd ..

/Users/joeranbosma/stack/Projects/M5Forecast


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, gc
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
os.environ['DATA_DIR'] = 'data/'
os.environ['SUB_DIR'] = 'submissions_uncertainty/'
for dirname, _, filenames in os.walk(os.environ['DATA_DIR']):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

data/.DS_Store
data/empty_pred.csv
data/pred_skeleton.csv
data/calendar.csv
data/sales_train_money.csv
data/sell_prices.csv
data/sales_train_validation.csv
data/sample_submission.csv
data/prep/converted_sales_level_12.csv
data/prep/converted_sales_level_11.csv
data/prep/converted_sales_level_10.csv
data/prep/converted_sales_all.csv
data/prep/norm_level_11.csv
data/prep/.DS_Store
data/prep/norm_level_10.csv
data/prep/norm_level_12.csv
data/prep/norm_level_1.csv
data/prep/norm_level_2.csv
data/prep/norm_level_3.csv
data/prep/norm_level_7.csv
data/prep/norm_level_6.csv
data/prep/converted_sales_level_9.csv
data/prep/norm_level_4.csv
data/prep/norm_level_5.csv
data/prep/converted_sales_level_8.csv
data/prep/converted_sales_level_5.csv
data/prep/norm_level_8.csv
data/prep/norm_level_9.csv
data/prep/converted_sales_level_4.csv
data/prep/converted_sales_level_6.csv
data/prep/converted_sales_level_7.csv
data/prep/converted_sales_level_3.csv
data/prep/norm_all.csv
data/prep/converted_sales_leve

In [3]:
from flow import load_data

In [4]:
calendar, sales_train_val, sell_prices = load_data()

In [5]:
df = calendar[(calendar['year'] == 2011)].loc[:, ['wday', 'month', 'snap_CA']]
df.head()

,wday,month,snap_CA
date,,,
2011-01-29,1,1,0
2011-01-30,2,1,0
2011-01-31,3,1,0
2011-02-01,4,2,1
2011-02-02,5,2,1


In [6]:
df['mu'] = df.apply(lambda row: 1.5 if (row.wday==2 or row.wday==3) else 0.75, axis=1)

In [7]:
df['sigma'] = df.apply(lambda row: 0.25 if (row.wday==2 or row.wday==3) else 0.13, axis=1)

In [8]:
quantiles = [0.005, 0.025, 0.165, 0.25, 0.5, 0.75, 0.835, 0.975, 0.995]

In [9]:
targets = df.apply(lambda row: pd.Series(np.random.normal(row.mu, row.sigma, size=1000)).quantile(quantiles), axis=1)
targets.head()

,0.005,0.025,0.165,0.25,0.5,0.75,0.835,0.975,0.995
date,,,,,,,,,
2011-01-29,0.429084,0.500587,0.628482,0.662077,0.743227,0.824528,0.861499,0.989619,1.086067
2011-01-30,0.834881,1.032595,1.265410,1.343565,1.503276,1.660740,1.728179,2.014386,2.143587
2011-01-31,0.838167,0.991256,1.246369,1.329488,1.508577,1.680692,1.763780,1.976137,2.167394
2011-02-01,0.424697,0.501162,0.629065,0.668797,0.758173,0.842714,0.878296,1.015888,1.090475
2011-02-02,0.438830,0.511583,0.630601,0.669403,0.754088,0.842727,0.883785,1.007879,1.084537


In [10]:
df = pd.concat((df, pd.get_dummies(df.wday, prefix='w')), axis=1)
df.drop(columns=['mu', 'sigma'], inplace=True)
df.head()

,wday,month,snap_CA,w_1,w_2,w_3,w_4,w_5,w_6,w_7
date,,,,,,,,,,
2011-01-29,1,1,0,1,0,0,0,0,0,0
2011-01-30,2,1,0,0,1,0,0,0,0,0
2011-01-31,3,1,0,0,0,1,0,0,0,0
2011-02-01,4,2,1,0,0,0,1,0,0,0
2011-02-02,5,2,1,0,0,0,0,1,0,0


In [11]:
df.to_csv("custom_layer/features.csv")
targets.to_csv("custom_layer/targets.csv")

## Generate dataset 2
Sample the distributions as above, but instead of providing the different quantiles, only provide one sample from its distribution. 

In [12]:
df = calendar.loc[:, ['wday', 'month', 'snap_CA']]
features = pd.concat((df, pd.get_dummies(df.wday, prefix='w')), axis=1)
features.head()

,wday,month,snap_CA,w_1,w_2,w_3,w_4,w_5,w_6,w_7
date,,,,,,,,,,
2011-01-29,1,1,0,1,0,0,0,0,0,0
2011-01-30,2,1,0,0,1,0,0,0,0,0
2011-01-31,3,1,0,0,0,1,0,0,0,0
2011-02-01,4,2,1,0,0,0,1,0,0,0
2011-02-02,5,2,1,0,0,0,0,1,0,0


In [13]:
df['mu'] = df.apply(lambda row: 1.5 if (row.wday==2 or row.wday==3) else 0.75, axis=1)
df['sigma'] = df.apply(lambda row: 0.25 if (row.wday==2 or row.wday==3) else 0.13, axis=1)
df.head()

,wday,month,snap_CA,mu,sigma
date,,,,,
2011-01-29,1,1,0,0.75,0.13
2011-01-30,2,1,0,1.50,0.25
2011-01-31,3,1,0,1.50,0.25
2011-02-01,4,2,1,0.75,0.13
2011-02-02,5,2,1,0.75,0.13


In [14]:
targets = df.apply(lambda row: np.random.normal(row.mu, row.sigma), axis=1)
targets.head()

date
2011-01-29    0.751272
2011-01-30    1.175314
2011-01-31    1.527411
2011-02-01    0.952662
2011-02-02    0.595021
dtype: float64

In [15]:
features.to_csv("custom_layer/features2.csv")
targets.to_csv("custom_layer/targets2.csv")